In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
data = [[110, 120, 130, 110, 115],
        [130, 130, 110,  90, 115],
        [110, 140, 130, 100,  95],
        [120, 110, 120, 120, 125],
        [100, 120, 150, 110, 105],
        [ 90, 100, 140,  80, 135]]

VEG = ['VEG1', 'VEG2']
NVEG = ['OIL1', 'OIL2', 'OIL3']
OILS = VEG + NVEG
MONTHS = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']
PREV_MONTH = {'JAN': 'INITIAL', 
              'FEB': 'JAN',
              'MAR': 'FEB',
              'APR': 'MAR',
              'MAY': 'APR',
              'JUN': 'MAY'}

costs = pd.DataFrame(data, columns=OILS, index=MONTHS)
hardness = dict(zip(OILS, [8.8, 6.1, 2.0, 4.2, 5.0]))

product_price  = 150
storage_cost   = 5
storage_limit  = 1000
veg_ref_limit  = 200
nveg_ref_limit = 250
upper_hardness = 6
lower_hardness = 3

In [3]:
model1 = gp.Model('Food Manufacture I')

# add vars
buy = model1.addVars(OILS, MONTHS,
                     name='buy')
use = model1.addVars(OILS, MONTHS,
                     name='use')
store = model1.addVars(OILS, MONTHS+['INITIAL'],
                       name='store')
for i in OILS:
    store[i,'INITIAL'].setAttr('ub', 500.0)
    store[i,'INITIAL'].setAttr('lb', 500.0)
    store[i,'JUN'].setAttr('ub', 500.0)
    store[i,'JUN'].setAttr('lb', 500.0)
    
produce = model1.addVars(MONTHS,
                         name='produce')

# objective function
model1.setObjective(gp.quicksum(product_price*produce[t] - 
                                gp.quicksum(costs[i][t]*buy[i,t]+storage_cost*store[i,t] for i in OILS) for t in MONTHS),
                    GRB.MAXIMIZE)

# add constraints
model1.addConstrs((gp.quicksum(use[i,t] for i in VEG) <= veg_ref_limit for t in MONTHS),
                  name='refine_veg')
model1.addConstrs((gp.quicksum(use[i,t] for i in NVEG) <= nveg_ref_limit for t in MONTHS),
                  name='refine_nveg')
model1.addConstrs((gp.quicksum(use[i,t] for i in OILS) - produce[t] == 0 for t in MONTHS),
                  name='mass_conservation')
model1.addConstrs((gp.quicksum(hardness[i]*use[i,t] for i in OILS) - upper_hardness*produce[t] <= 0 for t in MONTHS),
                  name='upper_hardness')
model1.addConstrs((gp.quicksum(hardness[i]*use[i,t] for i in OILS) - lower_hardness*produce[t] >= 0 for t in MONTHS),
                  name='lower_hardness')
model1.addConstrs((buy[i,t] + store[i,PREV_MONTH[t]] - use[i,t] - store[i,t] == 0 for i in OILS for t in MONTHS),
                  name='storage_link')
model1.addConstrs((store[i,t] <= 1000 for i in OILS for t in MONTHS),
                  name='storage_limit')
model1.update()

Academic license - for non-commercial use only - expires 2021-05-07
Using license file C:\Users\naraq\gurobi.lic


In [4]:
model1.write('Food Manufacture I.lp')
model1.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 90 rows, 101 columns and 288 nonzeros
Model fingerprint: 0xf8dc0d1c
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [5e+02, 5e+02]
  RHS range        [2e+02, 1e+03]
Presolve removed 36 rows and 16 columns
Presolve time: 0.01s
Presolved: 54 rows, 85 columns, 200 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8475779e+05   1.550637e+03   0.000000e+00      0s
      39    1.0784259e+05   0.000000e+00   0.000000e+00      0s

Solved in 39 iterations and 0.01 seconds
Optimal objective  1.078425926e+05


In [5]:
print('Profit of ${:.2f}'.format(model1.objVal))
produce_solution = pd.DataFrame([], columns=['PRODUCE'], index=MONTHS)
produce_solution = produce_solution.fillna(0)
for t in produce.keys():
    produce_solution['PRODUCE'][t] = produce[t].x
print('===PRODUCTION PLAN')
print(produce_solution)

buy_solution = pd.DataFrame([], columns=OILS, index=MONTHS)
buy_solution = buy_solution.fillna(0)
for i, t in buy.keys():
    buy_solution[i][t] = buy[i,t].x
print('===BUYING PLAN')
print(buy_solution)

use_solution = pd.DataFrame([], columns=OILS, index=MONTHS)
use_solution = use_solution.fillna(0)
for i, t in use.keys():
    use_solution[i][t] = use[i,t].x
print('===REFINING PLAN')
print(use_solution)

store_solution = pd.DataFrame([], columns=OILS, index=['INITIAL']+MONTHS)
store_solution = store_solution.fillna(0)
for i, t in store.keys():
    store_solution[i][t] = store[i,t].x
print('===STORAGE PLAN')
print(store_solution[1:])

Profit of $107842.59
===PRODUCTION PLAN
     PRODUCE
JAN      450
FEB      450
MAR      450
APR      450
MAY      450
JUN      450
===BUYING PLAN
     VEG1  VEG2  OIL1  OIL2  OIL3
JAN     0     0     0     0     0
FEB     0     0     0   250     0
MAR     0     0     0     0     0
APR     0     0     0     0     0
MAY     0     0     0     0   500
JUN   659   540     0   750     0
===REFINING PLAN
     VEG1  VEG2  OIL1  OIL2  OIL3
JAN    96   103     0   250     0
FEB   159    40     0   250     0
MAR    85   114     0     0   250
APR     0   200     0     0   250
MAY   159    40     0   250     0
JUN   159    40     0   250     0
===STORAGE PLAN
     VEG1  VEG2  OIL1  OIL2  OIL3
JAN   403   396   500   250   500
FEB   244   355   500   250   500
MAR   159   240   500   250   250
APR   159    40   500   250     0
MAY     0     0   500     0   500
JUN   500   500   500   500   500
